# Course project: cleavage plane for Ca doped YBCO

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from pwscf import make_struc_rt2, write_inputs

## Z Test

Make input files (from z-1 to z-4)

In [4]:
zs = np.arange(1,5)
for z in zs:
    [struc, struc_name] = make_struc_rt2(nxy = 1, nz = z, vacuum = 40)
    infile = write_inputs(struc = struc, dirname = 'Z_Test_LL', name = struc_name, calc = 'relax')

MAKING FILE
Writing /home/bond/Work/AP275/Z_Test_LL/YBCO_rt2_111_40vac_NOcleave_0sep_relax
MAKING FILE
Writing /home/bond/Work/AP275/Z_Test_LL/YBCO_rt2_112_40vac_NOcleave_0sep_relax
MAKING FILE
Writing /home/bond/Work/AP275/Z_Test_LL/YBCO_rt2_113_40vac_NOcleave_0sep_relax
MAKING FILE
Writing /home/bond/Work/AP275/Z_Test_LL/YBCO_rt2_114_40vac_NOcleave_0sep_relax


In [2]:
[struc, struc_name] = make_struc_undoped(nxy = 1, nz = 1, vacuum = 40)
infile = write_inputs(struc = struc, dirname = 'Z_Test_LL', name = struc_name, calc = 'relax')

MAKING FILE
Writing /home/bond/Work/AP275/Z_Test_LL/YBCO_rt2_111_40vac_NOcleave_0sep_relax


# Inital relax calculation (RUNS A CALC)

In [ ]:
def relax_YBCO(nk, ecut):
    pseudopots = {'Y': PseudoPotential(ptype='uspp', element='Y', functional='LDA', name='Y.pz-spn-rrkjus_psl.1.0.0.UPF'),
                  'Ba': PseudoPotential(ptype='uspp', element='Ba', functional='LDA', name='Ba.pz-spn-rrkjus_psl.1.0.0.UPF'),
                  'Cu': PseudoPotential(ptype='uspp', element='Cu', functional='LDA', name='Cu.pz-d-rrkjus.UPF'),
                  'O': PseudoPotential(ptype='uspp', element='O', functional='LDA', name='O.pz-rrkjus.UPF')}
    struc = make_struc_non_doped()
    kpts = Kpoints(gridsize=[nk, nk, nk], option='automatic', offset=True)
    dirname = 'YBCO_relax_ecut_{}_nk_{}'.format(ecut, nk)
    runpath = Dir(path=os.path.join(os.environ['WORKDIR'], "Project", dirname))
    input_params = PWscf_inparam({
        'CONTROL': {
            'calculation': 'relax',
            'pseudo_dir': os.environ['QE_POTENTIALS'],
            'outdir': runpath.path,
            'tstress': True,
            'tprnfor': True,
            'disk_io': 'none'
        },
        'SYSTEM': {
            'ecutwfc': ecut,
            'ecutrho': ecut * 12,
            'occupations': 'smearing',
            'smearing': 'mp',
            'degauss': 0.02
             },
        'ELECTRONS': {
            'diagonalization': 'david',
            'mixing_beta': 0.7,
            'conv_thr': 1e-7,
        },
        'IONS': {
            'ion_dynamics': 'bfgs'
        },
        'CELL': {},
        })

    output_file = run_qe_pwscf(runpath=runpath, struc=struc,  pseudopots=pseudopots,
                               params=input_params, kpoints=kpts, ncpu=16)
    output = parse_qe_pwscf_output(outfile=output_file)
    return output

def kpts_scan():
    ecut = 30
    nk = np.arange(1, 2)
    energy=[]
    for i in nk:
        output = relax_YBCO(ecut=ecut, nk=i)
        energy.append(output['energy'])
        print(i)
    return energy

In [ ]:
    lattice = numpy.array([[alat,0,0],[0,blat,0],[0,0,clat]])
    symbols = ['Ba', 'Ba', 'Y', 'Cu', 'Cu', 'Cu',
               'O', 'O', 'O', 'O','O','O','O']
    sc_pos = [[1.922334,    1.963076,    9.688204],
                [1.922334,    1.963076,    2.135460],
                [1.922334,    1.963076,    5.911832],
                [0.000000,    0.000000,    7.646103],
                [0.000000,    0.000000,    4.177560],
                [0.000000,    0.000000,    0.000000],
                [0.000000,    1.963076,    0.000000],
                [1.922334,    0.000000,    7.338392],
                [1.922334,    0.000000,    4.485271],
                [0.000000,    1.963076,    7.350192],
                [0.000000,    1.963076,    4.473472],
                [0.000000,    0.000000,    9.941573],
                [0.000000,    0.000000,    1.882091]]